CIDERは、ソースバッファ（cider-modeで動作）とREPLバッファの両方にインテリジェントなコード補完機能を提供します。
<!--
CIDER provides intelligent code completion for both source buffers (powered by cider-mode) and REPL buffers.
-->

### 注意

内部的にCIDERはClojureに対する[compliment](https://github.com/alexander-yakushev/compliment)とClojureScriptに対する[cljs-tooling](https://github.com/clojure-emacs/cljs-tooling)を利用します。
2つのライブラリーの改善は自動的にCIDERの改善につながります。
<!--
Note

Internally CIDER leverages compliment for Clojure and cljs-tooling for ClojureScript. Improvements to the two libraries automatically translate to improvements in CIDER.
-->

# 標準的な補完

すぐに使えるCIDERは、コード補完に標準のEmacsツールを使用します。
<kbd>TAB</kbd>か<kbd>M-TAB</kbd>を押すと、専用のバッファに補完候補が表示されます。
<!--
Out-of-the box CIDER uses the standard Emacs tooling for code completion. When you press TAB or M-TAB you'll get completion candidates in a dedicated buffer.
-->

# 自動補完

標準のEmacsツールはうまく機能しますが、CIDERユーザーは代わりに`company-mode`の使用を検討することをお勧めします。
`company-mode`は、ソースコードとREPLバッファの両方の自動補完に使用できます。`company-mode`をインストールするには：
<!--
While the standard Emacs tooling works just fine, we suggest that CIDER users consider using company-mode instead. Company mode can be used for auto-completion for both source code and REPL buffers. To install company-mode:
-->

<kbd>M-x</kbd> `package-install` <kbd>RET</kbd> `company` <kbd>RET</kbd>

インストール後、`company-mode`をグローバルに有効にすることができます。
<!--
After installation, you can turn on company-mode globally:
-->

```lisp
(global-company-mode)
```

または、モード固有のフックを通して、
<!--
or through mode-specific hooks:
-->

```lisp
(add-hook 'cider-repl-mode-hook #'company-mode)
(add-hook 'cider-mode-hook #'company-mode)
```

`company-mode`が有効になっていると、それは`cider-complete-at-point`から補完情報を受け取り、追加のセットアップやプラグインは不要です。
<!--
When company-mode is enabled, it will receive completion information from cider-complete-at-point and requires no additional setup or plugins.
-->

手動で補完を起動したい場合は、これを設定に追加できます。
<!--
If you'd prefer to trigger completions manually you can add this to your config:
-->

```lisp
(setq company-idle-delay nil) ; never start completions automatically
(global-set-key (kbd "M-TAB") #'company-complete) ; use M-TAB, a.k.a. C-M-i, as manual trigger
```

手動でインデントする機能を失うことなく<kbd>TAB</kbd>補完をさせるには、これをあなたの設定に追加することができます。
<!--
To make TAB complete, without losing the ability to manually indent, you can add this to your config:
-->

```lisp
(global-set-key (kbd "TAB") #'company-indent-or-complete-common)
```

# ファジィ候補マッチング

デフォルトでは、`company-mode`は、これまでに入力したものが実際に入力しようとしているものの接頭辞であるという前提で、補完候補を提示します。
たとえば、`map-`と入力すると、`map-`を名前の先頭に持つ補完候補のみが表示されます。
時々、あなたはあなたがタイプしたい項目の正確な接頭辞を知らない。
この場合、以下を追加することでCIDER特有の "ファジィ補完"を得ることができます。
<!--
By default company-mode will provide completion candidates with the assumption that whatever you've typed so far is a prefix of what you're really trying to type. For example, if you type map- then you'll only get completion candidates that have map- as the beginning of their names. Sometimes, you don't know the exact prefix for the item you want to type. In this case, you can get CIDER-specific "fuzzy completion" by adding:
-->

```lisp
(add-hook 'cider-repl-mode-hook #'cider-company-enable-fuzzy-completion)
(add-hook 'cider-mode-hook #'cider-company-enable-fuzzy-completion)
```
これで`company-mode`は接頭辞に対してマッチング候補を照合するときに特定のあいまいさを受け入れます。
たとえば、`mp`と入力すると、`map-indexed`が候補候補の1つとして表示され、`cji`は`clojure.java.io`を補完します。さまざまな補完例が[ここ](https://github.com/alexander-yakushev/compliment/wiki/Examples)に示されています。
<!--
Now, company-mode will accept certain fuzziness when matching candidates against the prefix. For example, typing mp will show you map-indexed as one of the possible completion candidates and cji will complete to clojure.java.io. Different completion examples are shown here.
-->

# 補完アノテーション

補完候補には、デフォルトでその種類と(文脈上の)名前空間に対応する省略形が付きます。
アノテーション(注釈)をフォーマットするために使用される関数は`cider-annotate-completion-function`によって設定できます。
使用される略語は`cider-completion-annotations-alist`によって設定され、それの名前空間が含まれるコンテキストは`cider-completion-annotations-include-ns`によって設定されます。
<!--
Completion candidates will be annotated by default with an abbreviation corresponding to their type, and (contextually) their namespace. The function used to format the annotation can be configured by cider-annotate-completion-function. The abbreviations used are configured by cider-completion-annotations-alist and the context in which their namespace is included is configured by cider-completion-annotations-include-ns.
-->

### メモ

補完アノテーションは、`cider-annotate-completion-candidates`を`nil`に設定することで無効にできます。
<!--
Tip

Completion annotations can be disabled by setting cider-annotate-completion-candidates to nil.
-->



# 状態クラスの更新とメソッドのキャッシュ

時には、REPLが起動した後に動的にロードされた依存関係を伴う新しいクラスを補完が認識できないことがあります(たとえばBoot経由)。
<kbd>M-x cider-completion-flush-caches</kbd>を実行する（またはメニュー「CIDER Interaction」-> 「Misc」-> 「Flush completion cache」の順に選択する）と、補完バックエンドはクラスパスで見つけられるすべてのクラスを再読み込みします。
<!--
Sometimes, the completion fails to recognize new classes that came with dependencies that were loaded dynamically after the REPL has started (e.g. via Boot). Executing M-x cider-completion-flush-caches (or going through the menu CIDER Interaction->Misc->Flush completion cache) forces the completion backend to re-read all classes it can find on the classpath.
-->